In [ ]:
# Import necessary libraries
import torch
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from tqdm import tqdm
import ultralytics
import os
import matplotlib.pyplot as plt
import numpy as np
np.random.seed(42)
import pandas as pd
import cv2
from IPython.display import Image
from ultralytics import YOLO

ultralytics.checks()


def render(image):
    if image.dtype == np.float64:
        image = cv2.convertScaleAbs(image)
    if len(image.shape) == 3 and image.shape[2] == 3: # BGR or RGB
        if np.array_equal(image[:, :, 0], image[:, :, 2]):
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    img_bytes = cv2.imencode('.png', image)[1].tobytes()
    display(Image(data=img_bytes))
    
    
def run_yolo(yolo, image_url, conf=0.25, iou=0.7):
    results = yolo(image_url, conf=conf, iou=iou)
    res = results[0].plot()[:, :, [2,1,0]]
    res = cv2.cvtColor(res, cv2.COLOR_BGR2RGB)
    return res

In [ ]:
yolo = YOLO('yolov8s.pt')
image_url = 'datasets/images/train/G0010117.JPG'
render(run_yolo(yolo, image_url))

In [ ]:

yolo.train(data='yolo_config.yaml', epochs=2)
valid_results = yolo.val()
print(valid_results)
yolo = YOLO('runs/detect/train/weights/best.pt')

render(run_yolo(yolo, image_url))